<a href="https://colab.research.google.com/github/auromaa-2004/Causal-AI/blob/main/Causal_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
path = kagglehub.dataset_download("varunchawla30/german-credit-data")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'german-credit-data' dataset.
Path to dataset files: /kaggle/input/german-credit-data


In [ ]:
import pandas as pd
import os

file_path = os.path.join(path, "german_credit_data.csv")
df = pd.read_csv(file_path)

df.head()

,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [ ]:
df = df.rename(columns={
    "laufkont": "checking_account_status",
    "laufzeit": "duration",
    "moral": "credit_history",
    "verw": "purpose",
    "hoehe": "credit_amount",
    "sparkont": "savings_account",
    "beszeit": "employment_duration",
    "rate": "installment_rate",
    "famges": "personal_status_sex",
    "buerge": "other_debtors",
    "wohnzeit": "present_residence",
    "verm": "property",
    "alter": "age",
    "weitkred": "other_installment_plans",
    "wohn": "housing",
    "bishkred": "number_of_existing_credits",
    "beruf": "job",
    "pers": "people_liable",
    "telef": "telephone",
    "gastarb": "foreign_worker",
    "kredit": "credit_risk"
})


In [ ]:
df.head()

,checking_account_status,duration,credit_history,purpose,credit_amount,savings_account,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_of_existing_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [ ]:
X = df.drop("credit_risk", axis=1)
y = df["credit_risk"]

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
# features only
X_all = df.drop(columns=["credit_risk"])
X_all_scaled = scaler.transform(X_all)

# predictions
preds = model.predict(X_all_scaled)
probs = model.predict_proba(X_all_scaled)[:, 0]  # P(approve)

# attach predictions
df_preds = df.copy()
df_preds["prediction"] = preds
df_preds["p_approve"] = probs

# borderline rejected candidates
borderline_rejected = (
    df_preds[df_preds["prediction"] == 1]
    .sort_values("p_approve", ascending=False)
)

borderline_rejected.head(10)


,checking_account_status,duration,credit_history,purpose,credit_amount,savings_account,employment_duration,installment_rate,personal_status_sex,other_debtors,...,other_installment_plans,housing,number_of_existing_credits,job,people_liable,telephone,foreign_worker,credit_risk,prediction,p_approve
656,2,24,4,3,1216,2,2,4,3,1,...,1,2,2,3,1,1,2,0,1,0.499537
988,1,18,2,0,976,1,2,1,2,1,...,3,2,1,2,2,1,2,0,1,0.498131
833,3,36,2,3,4210,1,3,4,3,1,...,3,2,1,3,2,1,2,0,1,0.494579
115,2,18,2,0,6260,1,4,3,3,1,...,3,1,1,2,2,1,2,1,1,0.494389
941,2,12,2,0,888,1,5,4,3,1,...,1,2,1,2,1,1,2,0,1,0.493456
908,1,24,3,0,1333,1,1,4,3,1,...,3,3,2,3,1,1,2,0,1,0.492222
769,1,15,2,4,1275,5,3,4,2,1,...,3,1,1,3,2,1,2,0,1,0.491238
850,2,45,2,3,3031,2,3,4,3,3,...,3,1,1,3,2,1,2,0,1,0.489994
617,1,18,2,0,2249,2,4,4,3,1,...,3,2,1,4,1,2,2,1,1,0.489180
902,2,36,3,0,2225,1,5,4,3,1,...,1,3,2,3,2,2,2,0,1,0.489128


In [ ]:
feature_cols = X.columns.tolist()

In [ ]:
# instance = borderline_rejected.iloc[[0]]   # keep as 1-row DataFrame

In [ ]:
top_k = 10

top_borderline = borderline_rejected.head(top_k)

instance = top_borderline.sample(1, random_state=None)


In [ ]:
mutable_features = [
    "credit_amount",
    "duration",
    "installment_rate",
    "checking_account_status",
    "savings_account",
    "other_debtors",
    "other_installment_plans"
]


In [ ]:
def build_search_space(df, instance):
    search_space = {}

    # Ensure scalar extraction
    credit_amt = instance["credit_amount"].item()
    duration_val = instance["duration"].item()

    # 1️⃣ credit_amount
    min_amt = int(df["credit_amount"].quantile(0.05))
    max_amt = int(credit_amt)

    search_space["credit_amount"] = list(
        range(min_amt, max_amt + 1, 100)
    )

    # 2️⃣ duration
    valid_durations = sorted(df["duration"].unique())
    search_space["duration"] = [
        d for d in valid_durations if d <= duration_val
    ]

    # 3️⃣ installment_rate
    search_space["installment_rate"] = sorted(
        df["installment_rate"].unique()
    )

    # 4️⃣ checking_account_status
    search_space["checking_account_status"] = sorted(
        df["checking_account_status"].unique()
    )

    # 5️⃣ savings_account
    search_space["savings_account"] = sorted(
        df["savings_account"].unique()
    )

    # 6️⃣ other_debtors
    search_space["other_debtors"] = sorted(
        df["other_debtors"].unique()
    )

    # 7️⃣ other_installment_plans
    search_space["other_installment_plans"] = sorted(
        df["other_installment_plans"].unique()
    )

    return search_space


In [ ]:
search_space = build_search_space(df, instance)

In [ ]:
best_cf = None
min_changes = float("inf")

for amount in search_space["credit_amount"]:
    for duration in search_space["duration"]:
        for rate in search_space["installment_rate"]:
            for checking in search_space["checking_account_status"]:
                for debtor in search_space["other_debtors"]:

                    cf = instance.copy()

                    cf.loc[:, "credit_amount"] = amount
                    cf.loc[:, "duration"] = duration
                    cf.loc[:, "installment_rate"] = rate
                    cf.loc[:, "checking_account_status"] = checking
                    cf.loc[:, "other_debtors"] = debtor

                    # constraints
                    if amount > instance["credit_amount"].iloc[0]:
                        continue
                    if duration > instance["duration"].iloc[0]:
                        continue

                    # ✅ ALIGN FEATURES + SCALE
                    cf_X = cf[feature_cols]
                    cf_X_scaled = scaler.transform(cf_X)

                    pred = model.predict(cf_X_scaled)[0]

                    if pred == 0:
                        changes = sum(
                            cf[f].iloc[0] != instance[f].iloc[0]
                            for f in search_space
                        )

                        if changes < min_changes:
                            min_changes = changes
                            best_cf = cf.copy()

if best_cf is not None:
    print("Counterfactual found")
    print(best_cf)
else:
    print("No counterfactual found")


Counterfactual found
     checking_account_status  duration  credit_history  purpose  \
902                        1        36               3        0   

     credit_amount  savings_account  employment_duration  installment_rate  \
902            708                1                    5                 4   

     personal_status_sex  other_debtors  ...  other_installment_plans  \
902                    3              1  ...                        1   

     housing  number_of_existing_credits  job  people_liable  telephone  \
902        3                           2    3              2          2   

     foreign_worker  credit_risk  prediction  p_approve  
902               2            0           1   0.489128  

[1 rows x 23 columns]


In [ ]:
def get_human_explanation(instance, cf):
    explanations = []

    for col in instance.columns:
        if col in ["credit_risk", "prediction", "p_approve"]:
            continue

        old = instance[col].iloc[0]
        new = cf[col].iloc[0]

        if old != new:
            explanations.append((col, old, new))

    return explanations


changes = get_human_explanation(instance, best_cf)
changes


[('checking_account_status', np.int64(2), np.int64(1)),
 ('credit_amount', np.int64(2225), np.int64(708))]

In [ ]:
feature_labels = {
    "credit_amount": "Loan amount",
    "duration": "Loan duration (months)",
    "installment_rate": "Installment rate",
    "checking_account_status": "Checking account status",
    "savings_account": "Savings account status",
    "other_debtors": "Guarantor / co-applicant"
}


In [ ]:
def explain_to_user(changes, labels):
    print("✅ Your loan could be approved if the following changes are made:\n")

    for feature, old, new in changes:
        if feature in labels:
            print(
                f"• {labels[feature]}: change from {old} → {new}"
            )

    print("\nℹ️ These are the smallest changes required to flip the decision.")


explain_to_user(changes, feature_labels)


✅ Your loan could be approved if the following changes are made:

• Checking account status: change from 2 → 1
• Loan amount: change from 2225 → 708

ℹ️ These are the smallest changes required to flip the decision.


In [ ]:
old_prob = model.predict_proba(
    scaler.transform(instance[feature_cols])
)[0][0]

new_prob = model.predict_proba(
    scaler.transform(best_cf[feature_cols])
)[0][0]

print(f"\nApproval probability increases from {old_prob:.2f} → {new_prob:.2f}")



Approval probability increases from 0.49 → 0.60


In [ ]:
!pip install dowhy


In [ ]:
from dowhy import CausalModel
import pandas as pd


In [ ]:
causal_df = df.copy()


In [ ]:
causal_graph = """
digraph {
    credit_amount -> credit_risk;
    duration -> credit_risk;
    installment_rate -> credit_risk;
    checking_account_status -> credit_risk;
    savings_account -> credit_risk;
    other_debtors -> credit_risk;

    credit_history -> credit_amount;
    credit_history -> credit_risk;

    employment_duration -> credit_risk;
    age -> credit_risk;
}
"""


In [ ]:
causal_model = CausalModel(
    data=causal_df,
    treatment="credit_amount",
    outcome="credit_risk",
    graph=causal_graph
)


ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...


In [ ]:
identified_estimand = causal_model.identify_effect()
print(identified_estimand)


Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
       d                                      
───────────────(E[credit_risk|credit_history])
d[creditₐₘₒᵤₙₜ]                               
Estimand assumption 1, Unconfoundedness: If U→{credit_amount} and U→credit_risk then P(credit_risk|credit_amount,credit_history,U) = P(credit_risk|credit_amount,credit_history)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!

### Estimand : 4
Estimand name: general_adjustment
Estimand expression:
       d                                      
───────────────(E[credit_risk|credit_history])
d[creditₐₘₒᵤₙₜ]                               
Estimand assumption 1, Unconfoundedness: If U→{credit_amount} and U→credit_risk then P(credit_risk|credit_amount,credit_history,U) = P(credit_risk|credit_amount,credit_history)



In [ ]:
estimate = causal_model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression"
)

print("Causal effect of credit_amount on credit_risk:")
print(estimate.value)


Causal effect of credit_amount on credit_risk:
-1.9368321046542292e-05


In [ ]:
causal_model_duration = CausalModel(
    data=causal_df,
    treatment="duration",
    outcome="credit_risk",
    graph=causal_graph
)

estimand_duration = causal_model_duration.identify_effect()

estimate_duration = causal_model_duration.estimate_effect(
    estimand_duration,
    method_name="backdoor.linear_regression"
)

print("Causal effect of duration on credit_risk:")
print(estimate_duration.value)


ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...


Causal effect of duration on credit_risk:
-0.005569700359914553


In [ ]:
# allow improvement over time
search_space["employment_duration"] = [
    v for v in df["employment_duration"].unique()
    if v >= instance["employment_duration"].iloc[0]
]

search_space["savings_account"] = [
    v for v in df["savings_account"].unique()
    if v >= instance["savings_account"].iloc[0]
]


In [ ]:
def counterfactual_existence_rate(df_rejected, model, scaler, feature_cols, build_search_space):
    count_cf = 0
    total = len(df_rejected)

    for i in range(min(50, total)):
        instance = df_rejected.iloc[[i]]
        search_space = build_search_space(df, instance)

        cf_found = False
        for amount in search_space["credit_amount"]:
            for duration in search_space["duration"]:
                for checking in search_space["checking_account_status"]:
                    cf = instance.copy()
                    cf.loc[:, "credit_amount"] = amount
                    cf.loc[:, "duration"] = duration
                    cf.loc[:, "checking_account_status"] = checking

                    cf_X = scaler.transform(cf[feature_cols])
                    if model.predict(cf_X)[0] == 0:
                        cf_found = True
                        break
                if cf_found:
                    break
            if cf_found:
                break

        if cf_found:
            count_cf += 1

    return count_cf / total


In [ ]:
def probability_lift(instance, cf, model, scaler, feature_cols):
    p_before = model.predict_proba(
        scaler.transform(instance[feature_cols])
    )[0][0]

    p_after = model.predict_proba(
        scaler.transform(cf[feature_cols])
    )[0][0]

    return p_after - p_before


In [ ]:
X_all = df.drop(columns=["credit_risk"])
X_all_scaled = scaler.transform(X_all)

preds = model.predict(X_all_scaled)

df_rejected = df[preds == 1]
print("Rejected applicants:", len(df_rejected))


Rejected applicants: 786


In [ ]:
cf_rate = counterfactual_existence_rate(
    df_rejected=df_rejected,
    model=model,
    scaler=scaler,
    feature_cols=feature_cols,
    build_search_space=build_search_space
)

print(f"Counterfactual Existence Rate: {cf_rate:.2f}")


Counterfactual Existence Rate: 0.01


In [ ]:
lift = probability_lift(
    instance=instance,
    cf=best_cf,
    model=model,
    scaler=scaler,
    feature_cols=feature_cols
)

print(f"Approval probability lift: {lift:.2f}")


Approval probability lift: 0.11
